In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install -q sentence_transformers


In [ ]:
!pip install -q llama-index


In [ ]:
%pip install -q llama-index-llms-huggingface

In [ ]:
import logging
import sys
from os import getenv


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext



In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate(
    "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
    "\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
!huggingface-cli login

In [ ]:
from llama_index.core import Settings

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,

    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)
Settings.llm = llm

In [ ]:
documents = SimpleDirectoryReader('/content/drive/MyDrive/data').load_data()

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator

import nest_asyncio

nest_asyncio.apply()

dataset_generator = RagDatasetGenerator.from_documents(
    documents=documents,
    llm=llm,
    num_questions_per_chunk=1,  # set the number of questions per nodes
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

In [ ]:
#after running this code. see how you can save this data set somewhere.